In [ ]:
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DualInputCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(DualInputCNN, self).__init__()

        # Jalur RGB
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, stride=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=1)
        self.dropout_conv1 = nn.Dropout(p=0.2)

        # Jalur LBP
        self.conv1_lbp = nn.Conv2d(1, 64, kernel_size=3, padding=1, stride=1)
        self.conv2_lbp = nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1)
        self.conv3_lbp = nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=1)
        self.dropout_conv2 = nn.Dropout(p=0.2)

        # Konvolusi setelah penggabungan dua jalur
        self.conv4 = nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1)
        self.dropout_conv3 = nn.Dropout(p=0.2)

        # Adaptive pooling dan fully connected layer
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc1 = nn.Linear(512 * 7 * 7, 512)
        self.dropout1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(512, 128)
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, rgb_input, lbp_input):
        # Forward jalur RGB
        x1 = F.relu(self.conv1(rgb_input))
        x1 = self.pool(x1)
        x1 = F.relu(self.conv2(x1))
        x1 = self.pool(x1)
        x1 = F.relu(self.conv3(x1))
        x1 = self.pool(x1)
        x1 = self.dropout_conv1(x1)

        # Forward jalur LBP
        x2 = F.relu(self.conv1_lbp(lbp_input))
        x2 = self.pool(x2)
        x2 = F.relu(self.conv2_lbp(x2))
        x2 = self.pool(x2)
        x2 = F.relu(self.conv3_lbp(x2))
        x2 = self.pool(x2)
        x2 = self.dropout_conv2(x2)

        # Gabungkan dua jalur
        x = torch.cat((x1, x2), dim=1)

        # Konvolusi gabungan
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = F.relu(self.conv5(x))
        x = self.pool(x)
        x = self.dropout_conv3(x)

        # Pooling, flatten, dan FC
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        return x

# Model instantiation
model = DualInputCNN(num_classes=2)
print(model)

DualInputCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout_conv1): Dropout(p=0.2, inplace=False)
  (conv1_lbp): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_lbp): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_lbp): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout_conv2): Dropout(p=0.2, inplace=False)
  (conv4): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout_conv3): Dropout(p=0.2, inplace=False)
  (adaptive_pool): AdaptiveAvgPool2d(output_size=(7, 7))
  (fc1): Linear(in_features=25088, out_features=512, bias=Tru

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import conf
conf.get_default().auth_token = "2xB12RQoSxsSJz5X5WzbIuFz8vU_3GhjFHPSyagvgGZk18NHc"

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch
import numpy as np
import pickle
import io
import os
from skimage.feature import local_binary_pattern
import torchvision.transforms as transforms

# ====== Setup Flask App ======
app = Flask(__name__)

# ====== Jalankan ngrok secara manual ======
public_url = ngrok.connect(5000)
print(" * Ngrok URL:", public_url)

# ====== Inisialisasi Device ======
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Inisialisasi MTCNN dan FaceNet ======
mtcnn = MTCNN(image_size=240, margin=0, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# ====== Load Anti-Spoofing Model dari Google Drive ======
MODEL_PATH = '/content/drive/MyDrive/dual_input_cnn_finetuned.pth'
antispoof_model = DualInputCNN(num_classes=2).to(device)
antispoof_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
antispoof_model.eval()

# ====== Load atau Inisialisasi Face Database ======
DB_PATH = '/content/drive/MyDrive/face_database.pkl'
if os.path.exists(DB_PATH):
    with open(DB_PATH, 'rb') as f:
        face_db = pickle.load(f)
else:
    face_db = {}

# ====== Fungsi LBP ======
def compute_lbp(pil_img):
    gray = pil_img.convert('L')
    np_gray = np.array(gray)
    lbp = local_binary_pattern(np_gray, P=8, R=1, method='uniform')
    lbp = (lbp - lbp.min()) / (lbp.max() - lbp.min())
    return lbp.astype(np.float32)

# ====== Preprocessing ======
transform_rgb = transforms.Compose([
    transforms.Resize((240, 240)),
    transforms.ToTensor()
])
transform_lbp = transforms.Compose([
    transforms.Resize((240, 240)),
    transforms.ToTensor()
])

# ====== Ekstraksi Embedding dari wajah yang sudah dicrop ======
def get_embedding(face_tensor):
    with torch.no_grad():
        return resnet(face_tensor.unsqueeze(0).to(device)).cpu().numpy()

# ====== Endpoint: Register Wajah ======
@app.route('/register_face', methods=['POST'])
def register_face():
    img = Image.open(request.files['image']).convert('RGB')
    face_tensor = mtcnn(img)
    if face_tensor is None:
        return jsonify({'status': 'fail', 'message': 'No face detected'})

    embedding = get_embedding(face_tensor)
    name = request.form['name']
    face_db[name] = embedding
    with open(DB_PATH, 'wb') as f:
        pickle.dump(face_db, f)
    return jsonify({'status': 'success', 'message': 'Face registered'})

# ====== Endpoint: Face Recognition ======
@app.route('/face_recognition', methods=['POST'])
def face_recognition():
    img = Image.open(io.BytesIO(request.files['image'].read())).convert('RGB')

    face_tensor = mtcnn(img)
    if face_tensor is None:
        return jsonify({'status': 'fail', 'message': 'Tidak ada wajah terdeteksi'})

    # Konversi tensor ke PIL untuk LBP
    face_pil = transforms.ToPILImage()(face_tensor.squeeze(0)).convert('RGB')

    # === Anti-Spoofing ===
    rgb_tensor = transform_rgb(face_pil).unsqueeze(0).to(device)
    lbp_np = compute_lbp(face_pil)
    lbp_tensor = torch.tensor(lbp_np).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        output = antispoof_model(rgb_tensor, lbp_tensor)
        pred = torch.argmax(output, dim=1).item()

    if pred != 0:
        return jsonify({'status': 'fail', 'message': 'Terdeteksi wajah palsu'})

    # === Ekstrak embedding wajah ===
    embedding = get_embedding(face_tensor)

    # === Pencocokan dengan database ===
    min_dist, recognized_name = float('inf'), None
    for name, db_embedding in face_db.items():
        dist = np.linalg.norm(embedding - db_embedding)
        if dist < min_dist:
            min_dist, recognized_name = dist, name

    if min_dist < 0.7:
        return jsonify({'status': 'success', 'name': recognized_name})
    else:
        return jsonify({'status': 'fail', 'message': 'Wajah pengguna tidak terdaftar'})

# ====== Jalankan Flask di thread terpisah ======
def run_flask():
    app.run(debug=False, use_reloader=False)

threading.Thread(target=run_flask).start()

 * Ngrok URL: NgrokTunnel: "https://6bb6-34-72-100-75.ngrok-free.app" -> "http://localhost:5000"


  0%|          | 0.00/107M [00:00<?, ?B/s]

 * Serving Flask app '__main__'


In [ ]:
'''import pickle
import numpy as np

# Path ke file database wajah
DB_PATH = '/content/drive/MyDrive/face_database.pkl'

# Load file
with open(DB_PATH, 'rb') as f:
    face_db = pickle.load(f)

# Tampilkan isi
print("Jumlah entri wajah:", len(face_db))
for name, embedding in face_db.items():
    print(f"Nama: {name}")
    print(f"Embedding shape: {embedding.shape}")
    print(f"Contoh nilai embedding (5 pertama): {embedding[0][:5]}")
    print("-" * 40)'''

 * Debug mode: off


'import pickle\nimport numpy as np\n\n# Path ke file database wajah\nDB_PATH = \'/content/drive/MyDrive/face_database.pkl\'\n\n# Load file\nwith open(DB_PATH, \'rb\') as f:\n    face_db = pickle.load(f)\n\n# Tampilkan isi\nprint("Jumlah entri wajah:", len(face_db))\nfor name, embedding in face_db.items():\n    print(f"Nama: {name}")\n    print(f"Embedding shape: {embedding.shape}")\n    print(f"Contoh nilai embedding (5 pertama): {embedding[0][:5]}")\n    print("-" * 40)'

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
'''import pickle

DB_PATH = '/content/drive/MyDrive/face_database.pkl'

# Load database
with open(DB_PATH, 'rb') as f:
    face_db = pickle.load(f)

# Hapus entri jika ada
target_name = 'Alvin'
if target_name in face_db:
    del face_db[target_name]
    print(f"{target_name} telah dihapus dari database.")
else:
    print(f"{target_name} tidak ditemukan dalam database.")

# Simpan kembali
with open(DB_PATH, 'wb') as f:
    pickle.dump(face_db, f)'''

'import pickle\n\nDB_PATH = \'/content/drive/MyDrive/face_database.pkl\'\n\n# Load database\nwith open(DB_PATH, \'rb\') as f:\n    face_db = pickle.load(f)\n\n# Hapus entri jika ada\ntarget_name = \'Alvin\'\nif target_name in face_db:\n    del face_db[target_name]\n    print(f"{target_name} telah dihapus dari database.")\nelse:\n    print(f"{target_name} tidak ditemukan dalam database.")\n\n# Simpan kembali\nwith open(DB_PATH, \'wb\') as f:\n    pickle.dump(face_db, f)'